# Open Graph Benchmark (OGB) Node Prediction for Micrososoft Academic Graph (OGBN-MAG)

The [Open Graph Benchmark (OGB)](https://ogb.stanford.edu/) is a collection of realistic, large-scale, and diverse benchmark datasets for machine learning on graphs.

This notebook demonstrate a [Graph Neural Network ([GNN])](https://en.wikipedia.org/wiki/Graph_neural_network) based on [TF-GNN](https://github.com/tensorflow/gnn) [OGBNMAG tutorial](https://colab.research.google.com/github/tensorflow/gnn/blob/master/examples/notebooks/ogbn_mag_e2e.ipynb), but using GoMLX.

The task [OGBN-MAG is described here](https://ogb.stanford.edu/docs/nodeprop/#ogbn-mag). This demo comes with a library that does the downloading, parsing and converting of the data to tensors for fast use.

The model is experimental, but it includes a basic GNN library that can be used for different projects.

**EXPERIMENTAL**, it has been used only for OGBN-MAG still.

See also [OGBN-MAG Leaderboard](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-papers100M) -- take with a grain of salt because different models use different tricks that may be considered leaking, or using extra data from the outside (so more data), and in some cases are very overfit to the task. But still, it's a fun dataset to work with.

See the subdirectory `demo` for a command line of the trainer (that can be run on the cloud somewhere) and it will save datapoints that can be plotted in the notebook later.


In [1]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx" "${HOME}/Projects/gonb" "${HOME}/Projects/gopjrt"
%goworkfix

	- Added replace rule for module "github.com/gomlx/gopjrt" to local directory "/home/janpf/Projects/gopjrt".
	- Added replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx".
	- Added replace rule for module "github.com/janpfeifer/gonb" to local directory "/home/janpf/Projects/gonb".


## Downloading Dataset

The method `mag.Download()` will download the dataset to the data directory, if not yet downloaded.
It then converts the dataset to tensors, which are then available for use.

The tensor is then saved for faster access. After saved, the next call to `mag.Download()` will take 1/2s.

In [9]:
import (
    "flag"
    mag "github.com/gomlx/gomlx/examples/ogbnmag"
    "github.com/janpfeifer/must"
	"github.com/gomlx/gomlx/backends"

    _ "github.com/gomlx/gomlx/backends/xla"
)

var (
    flagDataDir   = flag.String("data", "~/work/ogbnmag", "Directory to cache downloaded and generated dataset files.")
    backend = backends.New()
)

%%
start := time.Now()
must.M(mag.Download(*flagDataDir))
fmt.Printf("Elapsed: %s\n", time.Since(start))

Elapsed: 2.191164384s


## FNN model, No Graph

The first model will only use the paper features, and no relations. It serves as a baseline.

In a quick experiment, without much hyperparameter tuning, we got 27.27% on test accuracy, which is inline with the corresponding results in [the leaderboard](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-mag) (the Multi Layer Perceptron MLP entry in the bottom).

In [20]:
import (
	"github.com/gomlx/gomlx/ml/context"
    . "github.com/gomlx/gomlx/graph"
)
func config() (ctx *context.Context) {
    ctx = context.NewContext()
    ctx.RngStateReset()
    ctx.SetParams(map[string]any{
        "train_steps": 400_000, 
        "batch_size": 128,
        "optimizer": "adamw", 
        optimizers.LearningRateKey: 0.0001,
        layers.L2RegularizationKey: 1e-4,
        "normalization": "layer",
        "dropout": 0.1,
        "hidden_layers": 2,
        "num_nodes": 256,
        "plots": true,
    })
    return
}

In [12]:
%env GOMLX_BACKEND=xla:cpu

Set: GOMLX_BACKEND="xla:cpu"


In [ ]:
import "github.com/gomlx/gomlx/examples/ogbnmag/fnn"

%%
must.M(mag.Download(*flagDataDir))
ctx := config()
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "fnn-baseline"))
ctx.SetParam("num_checkpoints", 10)
ctx.SetParam("train_steps", 1000)
err := fnn.Train(backend, ctx)
if  err != nil {
    fmt.Printf("%+v\n", err)
}


loading: "checkpoint-n0000149-20240721-191922-step-00410512"
loading: "checkpoint-n0000147-20240721-191721-step-00404871"
loading: "checkpoint-n0000148-20240721-191821-step-00407634"
> restarting training from global_step=410512

Results on seeds_train:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L9.342704e+01,5.451585e+00 L1.398807e+02,3.573320e+01 L1.743645e+02,3.589330e+01 L2.022947e+02,5.718649e+01 L2.262080e+02,8.003493e+01 L2.473073e+02,8.769682e+01 L2.664046e+02,1.204028e+02 L2.840011e+02,1.433198e+02 L3.004240e+02,1.451495e+02 L3.158963e+02,1.505014e+02 L3.305756e+02,1.503185e+02 L3.446184e+02,1.604505e+02 L3.581316e+02,1.532689e+02 L3.711899e+02,1.692102e+02 L3.838458e+02,1.659167e+02 L3.961624e+02,1.727095e+02 L4.081817e+02,1.758886e+02 L4.199514e+02,1.819495e+02 L4.315009e+02,1.924932e+02 L4.428638e+02,1.793193e+02 L4.540588e+02,1.881934e+02 L4.651079e+02,1.930192e+02 L4.760213e+02,1.966786e+02 L4.868123e+02,2.073366e+02 L4.974946e+02,2.004981e+02 L5.080811e+02,1.994232e+02 L5.185829e+02,2.140379e+02 L5.290089e+02,2.082972e+02 L5.393657e+02,2.123226e+02 L5.496634e+02,2.080228e+02 L5.599078e+02,2.144039e+02 L5.701014e+02,2.189095e+02 L5.802524e+02,2.172628e+02 L5.903636e+02,2.198244e+02 L6.004377e+02,2.190696e+02 L6.104792e+02,2.179261e+02 L6.204905e+02,2.233237e+02 L6.304769e+02,2.186808e+02 L6.404399e+02,2.296362e+02 L6.503824e+02,2.236896e+02 L6.603053e+02,2.155703e+02 L6.702110e+02,2.289958e+02 L6.801008e+02,2.285612e+02 L6.899760e+02,2.377555e+02 L6.998382e+02,2.337073e+02 L7.096889e+02,2.214482e+02 L7.195285e+02,2.362460e+02 L7.293583e+02,2.408889e+02 L7.391793e+02,2.364976e+02 L7.489924e+02,2.337530e+02 L7.587978e+02,2.429473e+02 L7.685962e+02,2.392421e+02 L7.783886e+02,2.350109e+02 L7.881755e+02,2.420782e+02 L7.979572e+02,2.444797e+02 L8.077340e+02,2.413005e+02 L8.175067e+02,2.391964e+02 L8.272757e+02,2.463322e+02 L8.370411e+02,2.512724e+02 L8.468035e+02,2.432446e+02 L8.565630e+02,2.483906e+02 L8.663200e+02,2.395394e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,3.525736e+01 L9.342704e+01,3.567482e+01 L1.398807e+02,7.480942e+01 L1.743645e+02,7.503033e+01 L2.022947e+02,9.050226e+01 L2.262080e+02,1.047675e+02 L2.473073e+02,1.114667e+02 L2.664046e+02,1.249564e+02 L2.840011e+02,1.313555e+02 L3.004240e+02,1.370384e+02 L3.158963e+02,1.413791e+02 L3.305756e+02,1.445618e+02 L3.446184e+02,1.483647e+02 L3.581316e+02,1.511422e+02 L3.711899e+02,1.549359e+02 L3.838458e+02,1.600596e+02 L3.961624e+02,1.645374e+02 L4.081817e+02,1.691661e+02 L4.199514e+02,1.726947e+02 L4.315009e+02,1.764259e+02 L4.428638e+02,1.792125e+02 L4.540588e+02,1.837833e+02 L4.651079e+02,1.874673e+02 L4.760213e+02,1.898212e+02 L4.868123e+02,1.915870e+02 L4.974946e+02,1.954173e+02 L5.080811e+02,1.968601e+02 L5.185829e+02,2.006919e+02 L5.290089e+02,2.031159e+02 L5.393657e+02,2.065775e+02 L5.496634e+02,2.080447e+02 L5.599078e+02,2.105053e+02 L5.701014e+02,2.124798e+02 L5.802524e+02,2.135067e+02 L5.903636e+02,2.159475e+02 L6.004377e+02,2.170536e+02 L6.104792e+02,2.192552e+02 L6.204905e+02,2.209540e+02 L6.304769e+02,2.219763e+02 L6.404399e+02,2.236522e+02 L6.503824e+02,2.256100e+02 L6.603053e+02,2.264769e+02 L6.702110e+02,2.289147e+02 L6.801008e+02,2.304215e+02 L6.899760e+02,2.311711e+02 L6.998382e+02,2.322452e+02 L7.096889e+02,2.335722e+02 L7.195285e+02,2.352786e+02 L7.293583e+02,2.358759e+02 L7.391793e+02,2.381247e+02 L7.489924e+02,2.388210e+02 L7.587978e+02,2.407361e+02 L7.685962e+02,2.420113e+02 L7.783886e+02,2.430245e+02 L7.881755e+02,2.450874e+02 L7.979572e+02,2.461250e+02 L8.077340e+02,2.473103e+02 L8.175067e+02,2.490320e+02 L8.272757e+02,2.502630e+02 L8.370411e+02,2.511132e+02 L8.468035e+02,2.520578e+02 L8.565630e+02,2.529201e+02 L8.663200e+02,2.526794e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.356403e+01 L9.342704e+01,1.391886e+01 L1.398807e+02,4.252676e+01 L1.743645e+02,4.264503e+01 L2.022947e+02,4.860316e+01 L2.262080e+02,6.468864e+01 L2.473073e+02,7.287921e+01 L2.664046e+02,1.000530e+02 L2.840011e+02,1.239299e+02 L3.004240e+02,1.254231e+02 L3.158963e+02,1.310117e+02 L3.305756e+02,1.323422e+02 L3.446184e+02,1.443029e+02 L3.581316e+02,1.376203e+02 L3.711899e+02,1.504975e+02 L3.838458e+02,1.483242e+02 L3.961624e+02,1.574906e+02

## GNN (Graph Neural Networks)

To use GNNs we need first graphs to run the model on. The issue is, generally the graphs in real life are too large (social networks, relational datases, etc.), including OGNB-MAG.

So instead we use sampled sub-graphs to train. For inference we can also use sampled subgraphs, but later we show a work around where we can do inference, a layer of nodes at a time, where we don't need sampling.

### Sampling Sub-Graphs

We follow the same sampling strategy used in [TensorFlow GNN](https://github.com/tensorflow/gnn) library, describe it its [OGBN-MAG notebook](https://github.com/tensorflow/gnn/blob/main/examples/notebooks/ogbn_mag_e2e.ipynb).

The `magSampler` variable loads the definition of the data graph: its node and edge sets. The `magStrategy` defines how to sample from those nodes and edges. See it's specification in [gomlx/examples/ogbnmag/sampling.go](https://github.com/gomlx/gomlx/blob/main/examples/ogbnmag/sampling.go), it's only some 20 lines long.

The print out of the sampler and the strategy we are using:

In [4]:
%%
must.M(mag.Download(*flagDataDir))
magSampler := must.M1(mag.NewSampler(*flagDataDir))
magStrategy := mag.NewSamplerStrategy(magSampler, mag.BatchSize, mag.TrainSplit)
fmt.Printf("%s\n", magSampler)
fmt.Printf("\n%s\n", magStrategy)

Sampler: 4 node types, 8 edge types, Frozen
	NodeType "papers": 736,389 items
	NodeType "authors": 1,134,649 items
	NodeType "institutions": 8,740 items
	NodeType "fields_of_study": 59,965 items
	EdgeType "citedBy": ["papers"]->["papers"], 5,416,271 edges
	EdgeType "affiliatedWith": ["authors"]->["institutions"], 1,043,998 edges
	EdgeType "affiliations": ["institutions"]->["authors"], 1,043,998 edges
	EdgeType "hasTopic": ["papers"]->["fields_of_study"], 7,505,078 edges
	EdgeType "topicHasPapers": ["fields_of_study"]->["papers"], 7,505,078 edges
	EdgeType "writes": ["authors"]->["papers"], 7,145,660 edges
	EdgeType "writtenBy": ["papers"]->["authors"], 7,145,660 edges
	EdgeType "cites": ["papers"]->["papers"], 5,416,271 edges

Sampling strategy: (13 Rules)
> Rule "seeds": type=Node, nodeType="papers", Shape=(Int32)[128] (size=128), NodeSet.size=629571
  > Rule "citations": type=Edge, nodeType="papers", Shape=(Int32)[128 8] (size=1024), SourceRule="seeds", EdgeType="cites"
    > Rule "c

### Training Model

We use the vanilla GNN trainer and model defined in the `gnn` package.

In [74]:
import (
    "github.com/gomlx/gomlx/examples/ogbnmag/gnn"
)

func configGnn(baseDir string) *context.Context {
    must.M(mag.Download(baseDir))
    ctx := context.NewContext(manager)
    ctx.RngStateReset()
    
    stepsPerEpoch := mag.TrainSplit.Shape().Size() / mag.BatchSize + 1
    numEpochs := 10  // Taken from TF-GNN OGBN-MAG notebook.
    numTrainSteps := numEpochs * stepsPerEpoch
    
    ctx.SetParams(map[string]any{
        "train_steps": numTrainSteps,
        
        optimizers.ParamOptimizer: "adam", 
        optimizers.ParamLearningRate: 0.001,
        optimizers.ParamCosineScheduleSteps:  numTrainSteps,
        
        layers.ParamL2Regularization: 1e-5,
        layers.ParamDropoutRate: 0.2,

        mag.ParamEmbedDropoutRate: 0.0,
        
        gnn.ParamEdgeDropoutRate: 0.0,
        gnn.ParamNumGraphUpdates: 2,
        gnn.ParamReadoutHiddenLayers: 2,
        gnn.ParamPoolingType: "mean|sum",
        gnn.ParamUsePathToRootStates: false,
        
        "plots": true,
    })
    return ctx
}

%%
mag.BatchSize = 128  // Default is 128.
ctx := configGnn(*flagDataDir)
// ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-small_batch"))
// ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-use_path_to_root"))
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-reg_2"))
ctx.SetParam("num_checkpoints", 3)
ctx.SetParams(map[string]any{
    // "train_steps": 100,
    // "plots": false,
    // gnn.ParamEdgeDropoutRate: 0.1,
    // mag.ParamEmbedDropoutRate: 0.1,
    gnn.ParamNumGraphUpdates: 2,
    // gnn.ParamUsePathToRootStates: true,
    // gnn.ParamReadoutHiddenLayers: 2,
    layers.ParamDropoutRate: 0.25,
    layers.ParamL2Regularization: 3e-4,
})

// err := gnn.Train(ctx, *flagDataDir)
// if  err != nil {
//     fmt.Printf("%+v\n", err)
// }

In [14]:
%%
ctx := configGnn(*flagDataDir)
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-baseline_5"))

_, trainEvalDS, validEvalDS, testEvalDS := must.M4(gnn.MakeDatasets(*flagDataDir))
_, _, _ = trainEvalDS, validEvalDS, testEvalDS
must.M(gnn.Eval(ctx, *flagDataDir, trainEvalDS, validEvalDS, testEvalDS))

loading: "checkpoint-n0000171-20240229-012409-step-00049190"
Model in "/home/janpf/work/ogbnmag/gnn-baseline_5" trained for 49190 steps.
Results on train:
	Mean Loss+Regularization (#loss+): 1.542
	Mean Loss (#loss): 1.457
	Mean Accuracy (#acc): 57.49%
	elapsed 1m37.457905276s (train)
Results on valid:
	Mean Loss+Regularization (#loss+): 1.927
	Mean Loss (#loss): 1.841
	Mean Accuracy (#acc): 48.60%
	elapsed 11.140775268s (valid)
Results on test:
	Mean Loss+Regularization (#loss+): 1.922
	Mean Loss (#loss): 1.837
	Mean Accuracy (#acc): 48.10%
	elapsed 7.647081029s (test)


In [122]:
%%
ctx := configGnn(*flagDataDir)
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-baseline_17"))

_, trainEvalDS, validEvalDS, testEvalDS := must.M4(mag.MakeDatasets(*flagDataDir))
_, _, _ = trainEvalDS, validEvalDS, testEvalDS
must.M(mag.Eval(ctx, *flagDataDir, validEvalDS, testEvalDS))

loading: "checkpoint-n0000142-20240305-235708-step-00049190"
Model in "/home/janpf/work/ogbnmag/gnn-baseline_17" trained for 49190 steps.
Results on valid:
	Mean Loss+Regularization (#loss+): 1.920
	Mean Loss (#loss): 1.834
	Mean Accuracy (#acc): 48.64%
	elapsed 9.335698266s (valid)
Results on test:
	Mean Loss+Regularization (#loss+): 1.917
	Mean Loss (#loss): 1.830
	Mean Accuracy (#acc): 48.38%
	elapsed 5.863748141s (test)


In [7]:
import (
    stdplots "github.com/gomlx/gomlx/examples/notebook/gonb/plots"
    "github.com/gomlx/gomlx/examples/notebook/gonb/plotly"
)

func filterPoints(pt *stdplots.Point) bool {
    // Remove substrings
    for _, s := range []string{"Eval on "} {
        pt.MetricName = strings.Replace(pt.MetricName, s, "", -1)
    }
    // Replace substrings
    for _, pair := range [][2]string{
        {"Accuracy", "acc"}, {"Loss+Regularization", "loss+"}, {"Loss", "loss"}, {" Mean ", "#"}, {" Moving Average ", "~"}, 
        {"Train", "train"}, {"Validation", "valid"}, {"Test", "test"}, {": layer-wise eval", ":#acc"}} {
        pt.MetricName = strings.Replace(pt.MetricName, pair[0], pair[1], -1)
    }
    for _, exclude := range []string{ "Batch", "Train:~loss", "Train:~acc", "loss+" } {
        if strings.Index(pt.MetricName, exclude) != -1 { return false }
    }
    return true
}

func plotVersions(versions ...int) {
    plots := plotly.New()
    for _, version := range versions {
        // checkpoint := fmt.Sprintf("gnn-baseline_%d", version)
        checkpoint := fmt.Sprintf("small_%d", version)
        prefix := fmt.Sprintf("(%d) ", version)
        must.M(plots.LoadCheckpointData(path.Join(*flagDataDir, checkpoint),
            func (pt *stdplots.Point) bool {
                if !filterPoints(pt) { return false }
                pt.MetricName = prefix+pt.MetricName
                return true
            }))
    }
    plots.Plot()
}

%%
plotVersions(81, 82)

Metric: accuracy

Metric: loss